#nueral network
-comprise of layers/modules that perform operations on data
-torch.nn: provides all the building blocks
-is a module itself that consists of other modules
-can manage complex architectures

In [2]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


-every nn.Module implements the operations on input data in the forward method
*don't call model.forward() directly!

nn.Linear: output=input×weight+bias

nn.ReLU: positive -> unchanged, negative -> zero
-to introduce non-linearity
-can learn complex patterns in the data

nn.Flatten: multi-dimensional -> one-dimensional tensor

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


return
-dim=0: each output of 10 raw predicted values for each class
-dim=1: individual values of each output
-get prediction probabilities by passing it through an instance of the nn.Softmax module

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4643,  0.0798, -0.2263,  0.0140, -0.6367,  0.4089,  0.6212, -0.3229,
         -0.2898, -0.1462,  0.4078, -0.5249, -0.3491, -0.0913, -0.3321, -0.1647,
          0.0040,  0.1806, -0.1254, -0.7224],
        [-0.3164,  0.3085,  0.0751,  0.2143, -0.5903,  0.2747,  0.6774, -0.3052,
         -0.3078,  0.0981,  0.7680, -0.6163, -0.2142,  0.0019, -0.1263, -0.0028,
         -0.2351, -0.0881,  0.1794, -0.7882],
        [-0.3192,  0.0490,  0.0498, -0.0741, -0.6016,  0.2110,  0.3570, -0.1941,
         -0.1734, -0.0212,  0.3640, -0.2516, -0.1607,  0.0918,  0.1143, -0.1810,
         -0.0817,  0.0314, -0.1488, -0.7350]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0798, 0.0000, 0.0140, 0.0000, 0.4089, 0.6212, 0.0000, 0.0000,
         0.0000, 0.4078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0040, 0.1806,
         0.0000, 0.0000],
        [0.0000, 0.3085, 0.0751, 0.2143, 0.0000, 0.2747, 0.6774, 0.0000, 0.0000,
         0.0981, 0.7680, 0.0000, 0.0000, 0.0019, 0.00

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

-nn:Softmax: scale to values [0,1]. values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0204,  0.0194,  0.0191,  ...,  0.0005, -0.0217, -0.0072],
        [ 0.0010,  0.0275, -0.0230,  ..., -0.0097, -0.0116,  0.0240]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0010, -0.0185], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0358,  0.0044,  0.0276,  ..., -0.0344, -0.0165, -0.0409],
        [-0.0128,  0.0139,  0.0058,  ...,  0.0226,  0.0204, -0.0323]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Siz